In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [71]:
#filename = "/home/viviane/GravLens/Scraping/masterlens/masterlens_2.html"  # list with dfs from tables
filename = '/home/vivianemm/PycharmProjects/GravLens/Database/Scraping/masterlens/masterlens_1.html'
df_list = pd.read_html(filename)

In [72]:
# important dfs: 5,6,7,11,12,13,14,16,17,18,19,20,21
for i in df_list:
    print(i.columns.values)

[0]
[0 1 2 3]
[0 1 2 3 4 5]
[0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1 2 3]
[0 1]
[0 1 2 3]
[0 1 2 3 4 5 6 7]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]
[0 1 2 3 4 5]
[0 1 2 3 4 5 6 7 8]
[0 1]
[0 1]
[0 1 2 3 4 5 6 7 8]


In [73]:
df_list[12]

,0,1,2,3
0,System Name,SDSS J0008-0004,Discovery Date:,2008-08-01
1,Alternate Name(s),NaN,NaN,NaN


### Defining functions

In [74]:
def pick_table(df_list, key):
    for df in df_list:
        list_labels = list(df.columns.values)
        if any(isinstance(i, tuple) for i in list_labels):  # if labels is nested list
            labels = [item for t in list_labels for item in t]  # flatten
        else:
            labels = list_labels

        list_values = df.values.tolist()
        if any(isinstance(i, list) for i in list_values):  # if values is nested list
            values = [item for t in list_values for item in t]  # flatten
        else:
            values = list_values
              
        if key in values or key in labels:
            #df_list.remove(df)
            return df

In [75]:
from collections import defaultdict

def get_tables(soup, text=False):
    table_dict_list = []
    
    tables = soup.find_all('table')
    
    for table in tables:
        table_dict = defaultdict(list)
        
        
        headers = [header.get_text() for header in table.find_all('th')]
        
        if len(headers) == 0:
            pass
        else:
            
            if text:
                rows = [[data.get_text() for data in col.find_all('td')] for col in table.find_all('tr')]
            else:
                rows = [[data for data in col.find_all('td')] for col in table.find_all('tr')]

            for row in rows:
                columns_id = 0
                for data in row:
                    column_name = headers[columns_id]

                    table_dict[column_name].append(data)
                    columns_id += 1

            table_dict_list.append(table_dict)
        
    return table_dict_list

In [76]:
def drop_nans(df):
    df.dropna(axis=0, how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    return None

In [77]:
def split_col(df, old_new_list):  # old_new_list = [[oldname, newname], ...]
    for pair in old_new_list:
        if pair[0] in df.columns and type(list(df[pair[0]])[0]) == str and "±" in list(df[pair[0]])[0]:
            df[[pair[1], pair[1] + '_err']] = df[pair[0]].str.split("±", n = 1, expand = True)
            df.drop(pair[0], axis=1, inplace=True)
        else:
            pass
    return None

In [78]:
def get_table_links(table):
    href_list = []
    
    if isinstance(table, list):
        for row in table:
            href = row.find('a')['href']
            href_list.append(href)
    else:
        href_tags = table.find_all("a")
        for tag in href_tags:
            href = tag.get('href')
            href_list.append(href)
    return href_list

In [79]:
def get_links(soup, url=""):
    href_list = []
    
    if isinstance(soup, list):
        for row in soup:
            href = row.find('a')['href']
            href_list.append(url + href)
    else:
        href_tags = soup.find_all("a")
        for tag in href_tags:
            href = tag.get('href')
            href_list.append(url + href)
    return href_list

In [80]:
def save(df, filename, i=False):
    if len(df) > 0:
        #df.to_csv("/home/viviane/GravLens/Scraping/results/" + filename, index = i)
        df.to_csv('/home/vivianemm/PycharmProjects/GravLens/Database/Scraping/results/' + filename, index = i)
        
    return None

### System table
### (5, 6, 7, 11, 12, 20)

In [71]:
def system(df_list):
    # joining 5,6,7,11
    frames =[pick_table(df_list, 'Discovery'), pick_table(df_list, 'Lens Kind'), 
             pick_table(df_list,'Lens Grade'), pick_table(df_list, 'Description')]
    result_df = pd.concat(frames)
    result_df.reset_index(drop=True, inplace=True)
    system_df = result_df.drop(columns=0).T
    system_df.columns = list(result_df[0])
    
    
    # Splitting value ± error columns
    split_col(system_df, [['Einstein_R ["]','Einstein_R'], ['z_Lens', 'z_lens'], ['z_Source(s)', 'z_source'],
                          ['Stellar velocity disp', 'Stellar_v_disp']])
    
    # adding other 2 tables
    date_df = pick_table(df_list, 'Discovery Date:')
    n_df = pick_table(df_list, 'Number of Source Plane Images')
    
    system_df['Discovery Date'] = date_df[3][0]
    system_df['Name'] = date_df[1][0]
    system_df['N Images'] = n_df[1][0]
    system_df['Alternate Names'] = date_df[1][1]
    
    drop_nans(system_df)
    return system_df

In [73]:
system_df = system(df_list)
system_df

,Discovery,Lens Kind,Lens Grade,Einstein_R quality,z_Lens quality,z_Source quality,Description,Einstein_R,Einstein_R_err,z_lens,z_lens_err,z_source,z_source_err,Stellar_v_disp,Stellar_v_disp_err,Discovery Date,Name,N Images
1,SLACS,GAL-GAL,A,SIE model,spectroscopic,spectroscopic,Luminous red galaxy lensing three background o...,1.16,0.02,0.4400,0.0001,1.192,0.001,232,39,2008-08-01,SDSS J0008-0004,3


### Coordinates table (13)

In [23]:
def coordinates(df_list):
    coords_df = pick_table(df_list, 'Coordinates:') #.drop(2, axis=1)  # Manual coordinates
    coords_df.drop(2, axis=1, inplace=True)    
    coords_df = coords_df.set_index(0).T
    coords_df['Coordinates:'] = 'Manual'
    coords_df = coords_df.set_index('Coordinates:')

    return coords_df

In [24]:
coordinates(df_list)

KeyError: '[2] not found in axis'

### External links


In [958]:
def external(df_list, filename):    
    #external_df.columns = [''] * len(external_df.columns)
    #external_df.drop('External Links:.1', axis=1, inplace=True)
    
    soup = BeautifulSoup(open(filename), "lxml")
    external_df = pick_table(df_list, 'External Links:')
    for table in soup.find_all('table'):
            
        if table.th is not None and table.th.text == 'External Links:':
            refs = get_table_links(table)
        else:
            pass
            
            #if external_df is not None:
    external_df.columns = ['database', 'link']
    external_df = external_df.set_index('database')
    drop_nans(external_df)
    external_df['links'] = refs
    external_df.drop('link', axis=1, inplace=True)
    
    return external_df

### Flux table (SDSS 16, HST 17)

In [978]:
########### outros filtrosssss CFHTLS
def sdss(df_list):
    df = pick_table(df_list,'Filter')
    if df is not None and'SDSS' in df.columns.values:
        sdss_df = df.set_index('Band')
        sdss_df = sdss_df.drop(columns=['Unnamed: 1'])
    else:
        sdss_df = pd.DataFrame()

    return sdss_df


def hst(df_list):
    hst_df = pick_table(df_list,'HST')

    if hst_df is not None:
        hst_df = hst_df.drop(columns=['Unnamed: 1'])
        hst_df = hst_df.set_index('Band')
    else:
        hst_df = pd.DataFrame()
    return hst_df


def flux(df_list):
    sdss_df = sdss(df_list)
    hst_df = hst(df_list)

    flux_df = pd.concat([sdss_df, hst_df])
    #flux_df['release'] = list(sdss_df['SDSS']) + list(hst_df['HST'])
    #flux_df = flux_df.drop(columns=['SDSS','HST'])

    drop_nans(flux_df)
    split_col(flux_df, [['Lens Magnitude', 'lens_mag'], ['Flux [nmaggie]', 'Flux (nmaggie)'],
                    ['Reff [″]', "ref (arcsec)"], ['axis ratio (AB)', 'axis_ratio (AB)'],
                    ['PA [° E of N]','PA (deg)']])
    return flux_df

In [966]:
flux(df_list)    #CFHTLS

""


### Redshift table (18, 19)

In [899]:
def redshift(df_list):

    z_lens_df = pick_table(df_list, 'Lens Plane Images:')
    if z_lens_df is not None:
        z_lens_df = z_lens_df['Lens Plane Images:']
        #z_lens_df = z_lens_df.set_index('No.')
    else:
        z_lens_df = pd.DataFrame()

    z_source_df = pick_table(df_list, 'Source Plane Images:')
    if z_source_df is not None:
        z_source_df = z_source_df['Source Plane Images:']
        #z_source_df = z_source_df.set_index('No.')
    else:
        z_source_df = pd.DataFrame()

    z_df = pd.concat([z_lens_df, z_source_df], keys=['Lens', 'Source'])
    drop_nans(z_df)

    return z_df

In [900]:
z_lens_df = pick_table(df_list, 'Lens Plane Images:')
z_lens_df

Lens Plane Images:                             
                 No. Label     z   z_err Flux Mag
0                  1   NaN  0.44  0.0001  NaN NaN

In [901]:
redshift(df_list)

/home/viviane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



No.      z   z_err
Lens   0    1  0.440  0.0001
Source 0    1  1.192  0.0010
       1    2  1.192  0.0010
       2    3  1.192  0.0010

### Time delay table (21)

In [902]:
def time(df_list):
    time_df = pick_table(df_list, 'Time Delays:')

    if time_df is not None:
        times = list(time_df['Time Delays:']['Time Delay (days)'])
    #print(times)
        if np.isnan(times).all():
            time_df = None

    return time_df

In [903]:
d = time(df_list)
type(d)

NoneType

### References table (14)

In [904]:
def refs(filename, df_list):
    soup = BeautifulSoup(open(filename), "html.parser")

    for table in soup.find_all('table'):
    # links = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tb.find_all('a')]

        links = get_table_links(table)
        #print(links)

        if len(links) >= 2 and 'citation' in links[1]:
            papers = links[1::2]

    ref_df = pick_table(df_list, 'Discovery\xa0Paper')
    ref_df = ref_df.drop('Unnamed: 0', axis=1)

    ref_df['Links'] = papers
    ref_df.columns = ['Author', 'Title', 'Discovery', 'Links']
    return ref_df

In [905]:
refs(filename, df_list)

,Author,Title,Discovery,Links
0,Cardone et al. 2011 [MNRAS416(2011)1822],Secondary infall model and dark matter scaling...,NaN,http://admin.masterlens.org/citation.php?refer...
1,Auger et al. 2009 [ApJ705(2009)1099],"The Sloan Lens ACS Survey. IX. Colors, Lensing...",NaN,http://admin.masterlens.org/citation.php?refer...
2,Newton et al. 2009 [ApJ696(2009)1125],Enhanced Lensing Rate by Clustering of Massive...,NaN,http://admin.masterlens.org/citation.php?refer...
3,Grillo et al. 2009 [A&A501(2009)461],Photometric mass and mass decomposition in ear...,NaN,http://admin.masterlens.org/citation.php?refer...
4,Treu et al. 2009 [ApJ690(2009)670],The SLACS Survey. VIII. The Relation between E...,NaN,http://admin.masterlens.org/citation.php?refer...
5,Bolton et al. 2008 [ApJ682(2008)964],The Sloan Lens ACS Survey. V. The Full ACS Str...,Discovery Paper,http://admin.masterlens.org/citation.php?refer...


### Main code

In [27]:
for lens_n in range(1,51):
    filename = "/home/viviane/GravLens/Scraping/masterlens/masterlens_" + str(lens_n) + ".html"
    #filename = '/home/vivianemm/PycharmProjects/GravLens/Scraping/masterlens/masterlens_1.html'
    df_list = pd.read_html(filename)

    system_df = system(df_list)
    coords_df = coordinates(df_list)
    flux_df = flux(df_list)
    z_df = redshift(df_list)
    ref_df = refs(filename, df_list)
    external_df = external(df_list, filename)
    time_df = time(df_list)
    
    save(system_df, 'system_{}.csv'.format(lens_n))
    save(coords_df, 'coordinates_{}.csv'.format(lens_n))
    save(flux_df, 'flux_{}.csv'.format(lens_n), i=True)
    save(z_df, 'redshift_{}.csv'.format(lens_n), i=True)
    save(ref_df, 'references_{}.csv'.format(lens_n))
    save(external_df, 'external_{}.csv'.format(lens_n),i=True)
    
    print(lens_n)

ValueError: No tables found

In [1029]:
pd.read_csv('results/coordinates_50.csv')

,RA Hrs,RA Mins,RA Secs,RA [°],Dec Degrees,Dec Arcmin,Dec Arcsec,Dec [°]
0,10,4,34,151.143143,41,12,42.8,41.212109


### Castles


In [28]:
filename_c = '/home/vivianemm/PycharmProjects/GravLens/Database/castles.html'
df_list = pd.read_html(filename_c)

In [29]:
castles = df_list[1]
castles.rename(columns=castles.iloc[0], inplace=True)
castles.drop(['#', 'Image', 'G'], axis=1, inplace=True)
castles.drop([0], inplace=True)
castles = castles.set_index(['Lens Name'])
# split sigma
castles

,zs,zl,RA (J2000),Dec (J2000),E(B-V),ms (mag),ml (mag),FGHz (mJy),Nim,"size ("")",dt (days),sigma (km/s)
Lens Name,,,,,,,,,,,,
Q0047-2808,3.60,0.48,00:49:41.89,-27:52:25.7,0.016,NaN,I=20.05,NaN,4ER,2.7,NaN,229±15
HE0047-1756,1.66,0.41,00:50:27.83,-17:40:8.8,0.022,I=16.53/2,I=18.97,NaN,2,1.44,NaN,NaN
HST01247+0352,NaN,NaN,01:24:44.4,+03:52:00,0.029,I=24.13/2,I=21.86,NaN,2,2.20,NaN,NaN
HST01248+0351,NaN,NaN,01:24:45.6,+03:51:06,0.029,NaN,NaN,NaN,2,0.74,NaN,NaN
B0128+437,3.124,NaN,01:31:13.405,+43:58:13.14,0.082,NaN,NaN,F5=48,4,0.55,NaN,NaN
PMNJ0134-0931,2.216,0.77,01:34:35.67,-09:31:02.9,0.031,I=18.96/4,I=19.31,F5=529,5R,0.73,NaN,NaN
Q0142-100,2.72,0.49,01:45:16.5,-09:45:17,0.031,I=16.47/2,I=18.72,F5~1,2,2.24,NaN,NaN
QJ0158-4325,1.29,0.317,01:58:41.44,-43:25:04.20,0.015,I=17.39/2,I=18.91,F8<0.2,2,1.22,NaN,NaN
B0218+357,0.96,0.68,02:21:05.483,+35:56:13.78,0.068,I=19.28/2,I=20.06,F5=1209,2ER,0.34,10.5±0.4,NaN


In [85]:
castles.to_csv("castles.csv")

### Scraping Images

In [102]:
import requests
import shutil
import re

def get_image(soup, url, dir_path):
    
    image_list = []
    # Find all image on html code
    if isinstance(soup, list):
        for row in soup:
            image_url = row.find('img')
            image_list.append(image_url['src'])
    else:
        images_tags = soup.find_all("img")
        for image in images_tags:
            image_url = image['src']
            image_list.append(image_url)
    
    # Iter over each one
    for image in image_list:
        
        link = url + image
        
        # Download file
        filename = image.split('/')[-1]
        r = requests.get(link, stream=True)
        if r.status_code == 200:
            r.raw.decode_content = True
            with open(dir_path + filename, "wb") as f:
                shutil.copyfileobj(r.raw, f)
                print("[*] Downloaded Image: %s" % filename)
        else:
            print("[*] ERROR - Download Image: %s" % link)
    return 0

In [103]:
from bs4 import BeautifulSoup
import requests

page = requests.get('https://www.cfa.harvard.edu/castles/')
soup = BeautifulSoup(page.content, 'html.parser')


In [104]:
get_image(soup, 'https://www.cfa.harvard.edu/castles/', 'castlesimgs/')

[*] Downloaded Image: logo.jpg
[*] Downloaded Image: Q0047H.gif
[*] Downloaded Image: HE0047H.gif
[*] Downloaded Image: HST01247H.gif
[*] Downloaded Image: blank.gif
[*] Downloaded Image: B0128I.gif
[*] Downloaded Image: PMN0134H.gif
[*] Downloaded Image: Q0142H.gif
[*] Downloaded Image: CTQ414H.gif
[*] Downloaded Image: B0218H.gif
[*] Downloaded Image: HE0230I.gif
[*] Downloaded Image: SDSS0246H.gif
[*] Downloaded Image: CFRS03H.gif
[*] Downloaded Image: blank.gif
[*] Downloaded Image: MG0414K.gif
[*] Downloaded Image: HE0435H.gif
[*] Downloaded Image: B0445H.gif
[*] Downloaded Image: HE0512H.gif
[*] Downloaded Image: B0631H.gif
[*] Downloaded Image: B0712H.gif
[*] Downloaded Image: B0739H.gif
[*] Downloaded Image: blank.gif
[*] Downloaded Image: MG0751H.gif
[*] Downloaded Image: blank.gif
[*] Downloaded Image: HS0810H.gif
[*] Downloaded Image: HS0818H.gif
[*] Downloaded Image: APM08279K.gif
[*] Downloaded Image: B0850H.gif
[*] Downloaded Image: SDSS0903H.gif
[*] Downloaded Image: RXJ

0

In [106]:
import re

for lens in castles.index:
    name = lens.replace('+', '-')
    name = name.split('-')[0]
    link = 'https://www.cfa.harvard.edu/castles/Individual/{}.html'.format(name)
    
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        get_image(soup, link, 'castles_imgs_each/')
        print(link)
    except:
        print(link + '      nao')
        pass


https://www.cfa.harvard.edu/castles/Individual/Q0047.html
https://www.cfa.harvard.edu/castles/Individual/HE0047.html
https://www.cfa.harvard.edu/castles/Individual/HST01247.html
https://www.cfa.harvard.edu/castles/Individual/HST01248.html
https://www.cfa.harvard.edu/castles/Individual/B0128.html
https://www.cfa.harvard.edu/castles/Individual/PMNJ0134.html
https://www.cfa.harvard.edu/castles/Individual/Q0142.html
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/QJ0158.html/common/drupal/cfa_theme_logo_black.png
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/QJ0158.html /common/images/cleardot.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/QJ0158.html /common/images/cleardot.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/QJ0158.html /common/images/cleardot.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/QJ0158.html /common/images/cleardot.gi

[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html/common/drupal/cfa_theme_logo_black.png
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html /common/images/cleardot.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html /common/images/cleardot.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html /common/images/cleardot.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html /common/images/cleardot.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html /common/images/sections/about.jpg
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html/common/headers/footer-harvard_si.gif
[*] ERROR - Download Image: https://www.cfa.harvard.edu/castles/Individual/PMNJ1838.html//logs1.smithsonian.museum/dcsxkx59ubdzpxfcvjqxshx06_5r8l/njs.gif?d

In [112]:
page = requests.get('https://www.cfa.harvard.edu/castles/')
soup = BeautifulSoup(page.content, 'html.parser')

table = soup.find_all('table')[2]
#links_list = get_table_links(table)

#link = 'https://www.cfa.harvard.edu/castles/Individual/{}.html'.format(name)

#links_list
table.prettify

<bound method Tag.prettify of <table bgcolor="#FFFFFF" border="">
<caption>
<br/><!--
<tr bgcolor="#FF0080">
<td>Fooling around</td>
<td>Thorn &#222</td>
<td><font COLOR="FFFFFF">super<sup>1.41</sup></font></td>
<td>sub<sub>0.36</sub></td>
<td>strike <strike>16.7</strike></td>
<td>blink <blink>18.5 </blink></td>
<td>green <font COLOR="#00FF00">65.6</font></td>
<td>italic <i>(3)E<i></td>
<td bgcolor="#FFFFFF"><font size=+2>Big(+2) 6.1</font></td>
</tr>
--></caption>
<tr>
<th>#</th>
<!-- ImAgE --><th>Image</th>
<th>Binary Name</th>
<th>G</th>
<th>z<sub>s</sub></th>
<th>RA (J2000) </th>
<th>Dec (J2000) </th>
<th>E(B-V)</th>
<th>m<sub>tot</sub> (mag)</th>
<th>m<sub>diff</sub> (mag)</th>
<th>F<sub>GHz </sub>(mJy)</th>
<th>size (")</th>
</tr>
<tr align="CENTER">
<td>1</td>
<!-- ImAgE --><td><img align="CENTER" src="Postagestamps/Gifs/Thumbnails/blank.gif"/></td>
<td align="LEFT"><a href="Individual/LBQS0015.html">LBQS0015+0239</a></td>
<td>B</td>
<td>2.45</td>
<td align="left">00:18:11.37</t